Using Python to perform analysis on some production and manufacturing plant data.



The Head of Operations would like to track if the contract manufacturer is on track with the production plan and wants to see for each month
(i) the number of units manufactured.
(ii) the number of units planned
(iii) the production backlog 

##Question 1 - no of Units Manufactured

In [2]:
##each month of the manufactured units has a separate file, so we combine to have the full year data
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')
import pdb
from datetime import date
import psycopg2
from sqlalchemy import create_engine

##to merge the monthly data into one file
list_=[]
for report in glob.glob1(r"C:\Users\foyadeyi\Downloads\pcba cells data\data","*xlsx"):
    df = pd.read_excel(r"C:\Users\foyadeyi\Downloads\pcba cells data\data\%s"%report)
    df['Date']= pd.to_datetime(df.Date, format='%d-%m-%Y')
    df['Month'] = df.Date.dt.month
    df["Week"] = df["Date"].dt.week
    df["Week"]=df["Week"].apply(lambda x: f'W{x}')
    df['Type']='Manufactured'
    list_.append(df)
df_manuf = pd.concat(list_, ignore_index=True)
df_manuf


,Date,Product 1,Product 2,Month,Week,Type
0,2022-04-01,18,19,4,W13,Manufactured
1,2022-04-02,17,23,4,W13,Manufactured
2,2022-04-03,17,17,4,W13,Manufactured
3,2022-04-04,20,18,4,W14,Manufactured
4,2022-04-05,18,19,4,W14,Manufactured
...,...,...,...,...,...,...
360,2022-09-26,19,19,9,W39,Manufactured
361,2022-09-27,17,18,9,W39,Manufactured
362,2022-09-28,13,19,9,W39,Manufactured
363,2022-09-29,14,14,9,W39,Manufactured


In [4]:
#Groupby month to see how much was manufactured each month
df_group = df_manuf.groupby(['Month', 'Type'], as_index = False).sum()
df_group

,Month,Type,Product 1,Product 2
0,1,Manufactured,521,521
1,2,Manufactured,371,389
2,3,Manufactured,539,555
3,4,Manufactured,524,516
4,5,Manufactured,528,531
5,6,Manufactured,498,514
6,7,Manufactured,454,444
7,8,Manufactured,413,423
8,9,Manufactured,479,483
9,10,Manufactured,509,533


In [5]:
#Assigning the products into a single column
df_group1 = df_group[['Month', 'Product 1']]
df_group1['PN']='Product 1'
df_group1.rename(columns={'Product 1':'Manufactured'},inplace=True)

df_group2 = df_group[['Month', 'Product 2']]
df_group2['PN']='Product 2'
df_group2.rename(columns={'Product 2':'Manufactured'},inplace=True)

df_manufactured = pd.concat([df_group1,df_group2],ignore_index=True)
df_manufactured

,Month,Manufactured,PN
0,1,521,Product 1
1,2,371,Product 1
2,3,539,Product 1
3,4,524,Product 1
4,5,528,Product 1
5,6,498,Product 1
6,7,454,Product 1
7,8,413,Product 1
8,9,479,Product 1
9,10,509,Product 1


##Question 2 -- No of Units Planned

In [6]:
###dataframe for the production plan table, We assume there are two different table since the format is different from the manufacturing data
plan_list=[]
for report in glob.glob1(r"C:\Users\foyadeyi\Downloads\pcba cells data","*xlsx"):
    df_plan = pd.read_excel(r"C:\Users\foyadeyi\Downloads\pcba cells data\%s"%report)
    df_plan['Type']='Plan'
    df_plan['Week']=df_plan['Week'].str.replace('W', '').astype(int)
    plan_list.append(df_plan)
df_plan = pd.concat(plan_list, ignore_index=True)

year = 2022
start_date = pd.to_datetime(f'{year}-01-01')

# Define a function to map week numbers to month numbers
def week_to_month(week_number):
    # Calculate the date for the given week number
    week_date = start_date + pd.DateOffset(weeks=week_number - 1)
    return week_date.month

df_plan['Month'] = df_plan['Week'].apply(week_to_month)
df_plan = df_plan[['PN','Plan','Month']]
df_plan= df_plan.groupby(['Month','PN'], as_index = False).sum()
df_plan


,Month,PN,Plan
0,1,Product 1,600
1,1,Product 2,590
2,2,Product 1,460
3,2,Product 2,480
4,3,Product 1,470
5,3,Product 2,480
6,4,Product 1,590
7,4,Product 2,610
8,5,Product 1,460
9,5,Product 2,470


##Question 3 -- the Production Backlog

In [7]:
#Merge both data using left joins to see the corresponding plans for the manufactured Product 1 & 2
backlog = pd.merge(df_manufactured, df_plan, on=['PN', 'Month'],how='left')
backlog

,Month,Manufactured,PN,Plan
0,1,521,Product 1,600
1,2,371,Product 1,460
2,3,539,Product 1,470
3,4,524,Product 1,590
4,5,528,Product 1,460
5,6,498,Product 1,470
6,7,454,Product 1,500
7,8,413,Product 1,400
8,9,479,Product 1,480
9,10,509,Product 1,580


In [8]:
backlog['backlog'] = backlog.Plan - backlog.Manufactured
backlog

,Month,Manufactured,PN,Plan,backlog
0,1,521,Product 1,600,79
1,2,371,Product 1,460,89
2,3,539,Product 1,470,-69
3,4,524,Product 1,590,66
4,5,528,Product 1,460,-68
5,6,498,Product 1,470,-28
6,7,454,Product 1,500,46
7,8,413,Product 1,400,-13
8,9,479,Product 1,480,1
9,10,509,Product 1,580,71
